In [1]:
import pandas as pd
data_2012 = pd.read_csv('2012.csv')

/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
from sqlalchemy import create_engine

In [3]:
engine = create_engine('sqlite://', echo=False)

In [4]:
data_2012.head()

,Unnamed: 0,EMP,ESTAB,NAICS2012_TTL,OPTAX,GEO_TTL,NAICS2012,state,place
0,0,874,63,Total for all sectors,99,"Ituau county, American Samoa",00,60,37700
1,1,4344,164,Total for all sectors,99,"Ma'oputasi county, American Samoa",00,60,51300
2,2,0,5,Total for all sectors,99,"Sa'ole county, American Samoa",00,60,67300
3,3,0,4,Total for all sectors,99,"Sua county, American Samoa",00,60,72100
4,4,0,1,Total for all sectors,99,"Vaifanua county, American Samoa",00,60,86500


In [5]:
!curl https://raw.githubusercontent.com/fitnr/addfips/master/addfips/data/states.csv?accessType = Download > states.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1036  100  1036    0     0   3149      0 --:--:-- --:--:-- --:--:--  3158
curl: (6) Could not resolve host: =
curl: (6) Could not resolve host: Download


In [6]:
states = pd.read_csv('states.csv')

In [7]:
states.head()

,name,postal,fips
0,Alabama,AL,1
1,Alaska,AK,2
2,Arizona,AZ,4
3,Arkansas,AR,5
4,California,CA,6


In [8]:
states['fips'] = states['fips'].astype(str)
data_2012['state'] = data_2012['state'].astype(str)

In [9]:
states['fips'] = states.fips.str.zfill(2)
data_2012['state'] = data_2012.state.str.zfill(2)

In [10]:
states_cleaned = states[['postal','fips']]

In [11]:
states_cleaned.head()

,postal,fips
0,AL,01
1,AK,02
2,AZ,04
3,AR,05
4,CA,06


In [12]:
data_2012_us = data_2012.merge(states_cleaned, left_on = 'state', right_on = 'fips', how = 'inner')

In [13]:
data_2012_us.head()

,Unnamed: 0,EMP,ESTAB,NAICS2012_TTL,OPTAX,GEO_TTL,NAICS2012,state,place,postal,fips
0,0,874,63,Total for all sectors,99,"Ituau county, American Samoa",00,60,37700,AS,60
1,1,4344,164,Total for all sectors,99,"Ma'oputasi county, American Samoa",00,60,51300,AS,60
2,2,0,5,Total for all sectors,99,"Sa'ole county, American Samoa",00,60,67300,AS,60
3,3,0,4,Total for all sectors,99,"Sua county, American Samoa",00,60,72100,AS,60
4,4,0,1,Total for all sectors,99,"Vaifanua county, American Samoa",00,60,86500,AS,60


In [14]:
not_us_states = ['60','66','69','72','78']

In [15]:
data_2012_states = data_2012_us.loc[~data_2012_us['fips'].isin(not_us_states)]

In [16]:
data_2012_cities = data_2012_states[~data_2012_states['GEO_TTL'].str.contains('Balance')]

In [17]:
tax_codes = ['99','A']

In [18]:
data_2012_cities_all_occupation_1 = data_2012_cities.loc[data_2012_cities['OPTAX'].isin(tax_codes)]

In [19]:
data_2012_cities_remove_dupes = data_2012_cities_all_occupation_1.drop_duplicates() 

In [20]:
data_2012_cities_remove_dupes.head()

,Unnamed: 0,EMP,ESTAB,NAICS2012_TTL,OPTAX,GEO_TTL,NAICS2012,state,place,postal,fips
9561,2752938,0,2,Industrial equipment rental and leasing,99,Brookside CDP,5324902,10,9850,DE,10
9562,2752939,0,2,Industrial equipment rental and leasing,99,Claymont CDP,5324902,10,15310,DE,10
9563,2752940,0,1,Industrial equipment rental and leasing,99,Edgemoor CDP,5324902,10,23240,DE,10
9564,2752941,0,1,Industrial equipment rental and leasing,99,Elsmere town,5324902,10,24540,DE,10
9565,2752942,0,1,Industrial equipment rental and leasing,99,Newark city,5324902,10,50670,DE,10


In [21]:
data_2012_cities_remove_dupes['city_name'] = pd.np.where(data_2012_cities_remove_dupes.GEO_TTL.str.contains("city"),
                                                         data_2012_cities_remove_dupes.GEO_TTL.str.split("city"),
                                             pd.np.where(data_2012_cities_remove_dupes.GEO_TTL.str.contains("CDP"),
                                                         data_2012_cities_remove_dupes.GEO_TTL.str.split("CDP"),
                                             pd.np.where(data_2012_cities_remove_dupes.GEO_TTL.str.contains("town"),
                                                         data_2012_cities_remove_dupes.GEO_TTL.str.split("town"), 
                                                         "task")))

/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [22]:
data_2012_cities_remove_dupes['city_name'] = data_2012_cities_remove_dupes['city_name'].str[0]

/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [23]:
data_2012_cities_remove_dupes['fuck_this'] = data_2012_cities_remove_dupes['city_name'] + '_' + data_2012_cities_remove_dupes['postal']

/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [24]:
data_2012_cities_remove_dupes

,Unnamed: 0,EMP,ESTAB,NAICS2012_TTL,OPTAX,GEO_TTL,NAICS2012,state,place,postal,fips,city_name,fuck_this
9561,2752938,0,2,Industrial equipment rental and leasing,99,Brookside CDP,5324902,10,9850,DE,10,Brookside,Brookside _DE
9562,2752939,0,2,Industrial equipment rental and leasing,99,Claymont CDP,5324902,10,15310,DE,10,Claymont,Claymont _DE
9563,2752940,0,1,Industrial equipment rental and leasing,99,Edgemoor CDP,5324902,10,23240,DE,10,Edgemoor,Edgemoor _DE
9564,2752941,0,1,Industrial equipment rental and leasing,99,Elsmere town,5324902,10,24540,DE,10,Elsmere,Elsmere _DE
9565,2752942,0,1,Industrial equipment rental and leasing,99,Newark city,5324902,10,50670,DE,10,Newark,Newark _DE
9566,2752943,0,1,Industrial equipment rental and leasing,99,New Castle city,5324902,10,50800,DE,10,New Castle,New Castle _DE
9567,2752944,0,1,Industrial equipment rental and leasing,99,North Star CDP,5324902,10,52490,DE,10,North Star,North Star _DE
9569,2753202,0,1,Industrial equipment rental and leasing,99,Georgetown town,5324902,10,29090,DE,10,George,George_DE
9572,2755027,0,4,Lessors of nonfinancial intangible assets (exc...,99,Newark city,533,10,50670,DE,10,Newark,Newark _DE
9573,2755028,0,1,Lessors of nonfinancial intangible assets (exc...,99,Pike Creek CDP,533,10,56490,DE,10,Pike Creek,Pike Creek _DE


In [25]:
jobs = list(data_2012['NAICS2012_TTL'].unique())

In [26]:
for i in jobs:
    if 'engineering' in i:
        print(i)
    if 'science' in i:
        print(i)
    if 'lawyer' in i:
        print(i)
    if 'medical' in i:
        print(i)
    if 'medicine' in i:
        print(i)
    if 'education' in i:
        print(i)    
    if 'computer' in i:
        print(i)     
    if 'accounting' in i:
        print(i)    
    if 'engineer' in i:
        print(i)
    if 'tax' in i:
        print(i)
    if 'technology' in i:
        print(i)
    if 'programming' in i:
        print(i)
    if 'artists' in i:
        print(i)
    if 'music' in i:
        print(i)
    if 'bio' in i:
        print(i)
    if 'research' in i:
        print(i)
    if 'legal' in i:
        print(i)
    if 'development' in i:
        print(i)
    if 'planning' in i:
        print(i)
    if 'surgical' in i:
        print(i)

Heavy and civil engineering construction
Heavy and civil engineering construction
Veneer, plywood, and engineered wood product manufacturing
Pharmaceutical and medicine manufacturing
Electronic computer manufacturing
Computer terminal and other computer peripheral equipment manufacturing
Navigational, measuring, electromedical, and control instruments manufacturing
Electromedical and electrotherapeutic apparatus manufacturing
Surgical and medical instrument manufacturing
Computer and computer peripheral equipment and software merchant wholesalers
Computer and computer peripheral equipment merchant wholesalers
Surgical, medical, and hospital supplies merchant wholesalers
Sporting goods, hobby, musical instrument, and book stores
Sporting goods, hobby, and musical instrument stores
Other water passenger transportation (including water taxi)
Cable and other subscription programming
Direct life, health, and medical insurance carriers
Direct health and medical insurance carriers
Direct insu

In [27]:
creative_class = ['Heavy and civil engineering construction',
'Veneer, plywood, and engineered wood product manufacturing',
'Pharmaceutical and medicine manufacturing',
'Electronic computer manufacturing',
'Computer terminal and other computer peripheral equipment manufacturing',
'Navigational, measuring, electromedical, and control instruments manufacturing',
'Electromedical and electrotherapeutic apparatus manufacturing',
'Surgical and medical instrument manufacturing',
'Computer and computer peripheral equipment and software merchant wholesalers',
'Computer and computer peripheral equipment merchant wholesalers',
'Surgical, medical, and hospital supplies merchant wholesalers',
'Direct life, health, and medical insurance carriers',
'Direct health and medical insurance carriers',
'Direct insurance (except life, health, and medical) carriers',
'Offices of lawyers',
'Legal aid societies and similar legal services',
'Other legal services',
'All other legal services',
'Accounting, tax preparation, bookkeeping, and payroll services',
'Other accounting services',
'Architectural, engineering, and related services',
'Custom computer programming services',
'Other computer related services',
'Scientific research and development services',
'Research and development in the physical, engineering, and life sciences',
'Research and development in the physical, engineering, and life sciences',
'Research and development in biotechnology',
'Research and development in the physical, engineering, and life sciences (except biotechnology)',
'Research and development in the physical and engineering sciences',
'Research and development in other life sciences',
'Research and development in the social sciences and humanities',
'Marketing research and public opinion polling',
'Economic or industrial planning or development organization',
'Business schools and computer and management training',
'Professional and management development training',
'Art, drama, and music schools',
'Family planning centers',
'HMO medical centers',
'Freestanding ambulatory surgical and emergency centers',
'General medical and surgical hospitals',
'General medical and surgical hospitals, government',
'General medical and surgical hospitals, except government',
'Residential intellectual and developmental disability, mental health, and substance abuse facilities',
'Residential intellectual and developmental disability facilities',
'Musical groups and artists',
'Symphony orchestras and chamber music organizations',
'Other music groups and artists',
'Agents and managers for artists, athletes, entertainers, and other public figures',
'Independent artists, writers, and performers']

In [28]:
data_2012_cities_remove_dupes['class'] = pd.np.where(
     data_2012_cities_remove_dupes['NAICS2012_TTL'].isin(creative_class), 
    '1_creative',  
    pd.np.where(~data_2012_cities_remove_dupes['NAICS2012_TTL'].isin(creative_class), 
    '2_regular','3_none'
     ))

/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [29]:
data_2012_cities_remove_dupes['class'].unique()

array(['2_regular', '1_creative'], dtype=object)

In [30]:
data_2012_cleaned_af = data_2012_cities_remove_dupes.reset_index()

In [31]:
data_2012_cleaned_af.to_csv('2012_semi_processed.csv')

In [32]:
data_2012_cleaned_af = data_2012_cleaned_af[['fuck_this','EMP','ESTAB','class']]

In [33]:
data_2012_cleaned_af['class'].unique()

array(['2_regular', '1_creative'], dtype=object)

In [34]:
econ_data_final = data_2012_cleaned_af.groupby(['fuck_this','class']).sum().unstack()

In [35]:
econ_data_final.head()

EMP                ESTAB          
class                       1_creative 2_regular 1_creative 2_regular
fuck_this                                                            
Abbeville _AL                      0.0     522.0       12.0     147.0
Abbeville _GA                      NaN      84.0        NaN      20.0
Abbeville _LA                    288.0    2427.0      120.0     679.0
Abbeville _SC                     33.0      58.0       20.0     187.0
Aberdeen Proving Ground _MD        0.0    5322.0       35.0     381.0

In [36]:
econ_data_final = econ_data_final.reset_index()
econ_data_final.columns = ['city_state','creative_employees','regular_employees',
                           'creative_establishments','regular_establishments']

In [37]:
#econ_data_final = econ_data_final.fillna(0)

In [38]:
#econ_data_final['percent_creative_class'] = (econ_data_final['creative_employees']/(econ_data_final['creative_employees']+econ_data_final['regular_employees']))*100

In [39]:
#econ_data_final.sort_values(by=['percent_creative_class'],  ascending=False)

In [40]:
econ_data_final.to_csv('2012_creative_class.csv')